<a href="https://colab.research.google.com/github/sumankmaiti/NIP-Neuron-Label-Inverse-Perturbation/blob/main/NIP_pytorch_trial2_grad_and_output_of_flatten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cleverhans

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

from absl import app, flags
from easydict import EasyDict
import torchvision
from google.colab.patches import cv2_imshow
import cv2

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)

In [ ]:
import torchvision.models as models

pretrained_model = models.vgg19(pretrained=True)

print(pretrained_model)

In [ ]:
pretrained_size = 224
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds = [0.229, 0.224, 0.225]

# train_transforms = transforms.Compose([
#                            transforms.Resize(pretrained_size),
#                            transforms.RandomRotation(5),
#                            transforms.RandomHorizontalFlip(0.5),
#                            transforms.RandomCrop(pretrained_size, padding=10),
#                            transforms.ToTensor(),
#                            transforms.Normalize(mean=pretrained_means,
#                                                 std=pretrained_stds)
                      #  ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=pretrained_means,
                                                std=pretrained_stds)
                       ])

In [ ]:
ROOT = '.data'

# train_data = datasets.CIFAR10(ROOT,
#                               train=True,
#                               download=True,
#                               transform=train_transforms)

test_data = datasets.CIFAR10(ROOT,
                             train=False,
                             download=True,
                             transform=test_transforms)

# print(f'Number of training examples: {len(train_data)}')
# print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
BATCH_SIZE = 8

# train_iterator = data.DataLoader(train_data,
#                                  shuffle=True,
#                                  batch_size=BATCH_SIZE)

# valid_iterator = data.DataLoader(valid_data,
#                                  batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Store the output of a layer

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# def get_predictions(model, iterator):

pretrained_model.eval().to(device)

    # images = []
original_labels = []
probs = []
predected_labels = []

# with torch.no_grad():

for (x, y) in tqdm(test_iterator):

    x = x.to(device)
    # print(x.shape)

    # get activation of flatten layer
    pretrained_model.classifier[0].register_forward_hook(get_activation('flatten'))

    y_pred = pretrained_model(x)

    print(activation['flatten'].shape)       

    # get gradient og flatten layer
    loss = criterion(y_pred, y)
    loss.backward()
    flatten_layer_grad = pretrained_model.classifier[0].weight.grad
    print(flatten_layer_grad.shape)
    
    y_pred = F.softmax(y_pred, dim=-1) # column wise softmax
    # print(y_pred.shape, y_pred)
    prob, class_idx= y_pred.max(1) # returns probabilities and its class id
    # print(class_idx)
    # print(probs)

    # images.append(x.cpu())
    original_labels.append(y.cpu())
    probs.append(prob.cpu())
    predected_labels.append(class_idx.cpu())
    print(predected_labels)

    # images = torch.cat(images, dim=0)  # concatinate each batch to produce one image set
original_labels = torch.cat(original_labels, dim=0) 
probs = torch.cat(probs, dim=0)
predected_labels = torch.cat(predected_labels, dim = 0)

    # return labels, probs

Perturbed image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Evaluate on clean and adversarial data
pretrained_model.eval().to(device)

adv_img = []
for (x, y) in tqdm(test_iterator):

  x, y = x.to(device), y.to(device)

  x_fgm = fast_gradient_method(pretrained_model, x, 0.3,2)
  adv_img.append(x_fgm)
    # x_pgd = projected_gradient_descent(net, x, 0.3, 0.01, 50, 2)

    
    # f, axarr = plt.subplots(1,3)
    # axarr[0].imshow(x[10].cpu().detach().permute(1, 2, 0))
    # axarr[1].imshow(x_fgm[10].cpu().detach().permute(1, 2, 0))
    # axarr[2].imshow(x_pgd[10].cpu().detach().permute(1, 2, 0))
    
  # _, y_pred = pretrained_model(x).max(1)  # model prediction on clean examples
  pretrained_model.classifier[0].register_forward_hook(get_activation('flatten_adv'))
    
  _, y_pred_fgm = pretrained_model(x_fgm).max(1)  # model prediction on FGM adversarial examples
  print(activation['flatten_adv'].shape)    # print(y_pred.shape)
 
  print(y_pred_fgm)
    # # _ , y_pred_pgd  =  net ( x_pgd ). max (1)
    #   print("Original label:",y, "\nPredected label without attack:", y_pred, "\nPredected label after FGM attack:" , y_pred_fgm,  "\nPredected label after PGD attack:", y_pred_pgd)
adv_img = torch.cat(adv_img, dim = 0)
